# L2
- Apply LOWESS to all

In [52]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr

%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.pandas  # noqa


#
import pynsitu as pyn
from pynsitu.maps import crs

import dask.dataframe as dd
from lib import raw_dir, root_dir, KEYS

In [2]:
# drifters
campaign = "drifters_CSWOT_BIOSWOT_UWA"
yaml = f"{campaign}.yaml"

cp = pyn.Campaign(os.path.join(root_dir, yaml))
KEYS

['carthe_cnr',
 'carthe_lops',
 'code_ogs',
 'svp_ogs',
 'svp_scripps',
 'svp_shom',
 'svp_bcg',
 'spotter_lops',
 'carthe_uwa',
 'melodi_eodyn']

In [3]:
# CHOOSE HERE
key = KEYS[0]

path = glob(os.path.join(raw_dir, "L1_" + key + "*"))[0]
df = pd.read_csv(path, parse_dates=["time"], dtype={"id": "string"}).set_index("id")
ids = sorted(list(df.index.unique()))
# df = df.repartition(8).reset_index().persist()


ids_map = {p: cp[p]["serial_number"] for p in cp if key in p}
ids_yaml = sorted(set([id for _, id in ids_map.items()]))
ids_imap = {v: k for k, v in ids_map.items()}

print("drifter id's in data file:")
print(ids)
print("drifter id's in yaml file:")

print(ids_yaml)

flag = set(ids).issuperset(ids_yaml)
if flag:
    print("Data file and campaign yaml file agree upon drifter ids")
else:
    assert False, "Data file and campaign yaml file do not agree upon drifter ids"

drifter id's in data file:
['0-4388553', '0-4388559', '0-4388587', '0-4388589', '0-4388593', '0-4388625', '0-4388632', '0-4388634', '0-4388636', '0-4388640', '0-4388923', '0-4388926', '0-4388938', '0-4388942', '0-4388943', '0-4389009', '0-4389089', '0-4389439', '0-4389458', '0-4389467']
drifter id's in yaml file:
['0-4388553', '0-4388559', '0-4388587', '0-4388589', '0-4388593', '0-4388625', '0-4388632', '0-4388634', '0-4388636', '0-4388640', '0-4388923', '0-4388926', '0-4388938', '0-4388942', '0-4388943', '0-4389009', '0-4389089', '0-4389439', '0-4389458', '0-4389467']
Data file and campaign yaml file agree upon drifter ids


In [5]:
df

,time,GpsQuality,lat,lon,BatteryStatus,GPS Confidence,GPS Fail Counter,deployment,platform,x,y,dt,velocity_east,velocity_north,velocity,acceleration_east,acceleration_north,acceleration
id,,,,,,,,,,,,,,,,,,
0-4388553,2023-04-23 21:55:10,3,40.868990,4.196691,GOOD,0,0,d0,carthe_cnr_00,-304389.826561,306083.175744,NaN,0.283761,0.267734,0.390130,NaN,NaN,NaN
0-4388553,2023-04-23 22:04:58,3,40.870127,4.198408,GOOD,0,0,d0,carthe_cnr_00,-304240.032180,306203.639071,588.0,0.227157,0.187953,0.294833,-6.207149e-05,-0.000104,0.000121
0-4388553,2023-04-23 22:14:59,3,40.870996,4.199889,GOOD,0,0,d0,carthe_cnr_00,-304111.332710,306295.145480,601.0,0.210857,0.144870,0.255828,-1.973650e-05,-0.000064,0.000067
0-4388553,2023-04-23 22:24:59,3,40.871694,4.201412,GOOD,0,0,d0,carthe_cnr_00,-303979.865485,306367.531446,600.0,0.203439,0.111238,0.231865,-1.619346e-05,-0.000039,0.000042
0-4388553,2023-04-23 22:34:59,3,40.872198,4.202785,GOOD,0,0,d0,carthe_cnr_00,-303861.911056,306418.886055,600.0,0.191425,0.098282,0.215181,-2.534735e-05,-0.000002,0.000025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0-4389467,2023-06-23 01:46:35,3,38.347274,4.413908,LOW,0,0,d0,carthe_cnr_19,69048.107005,-89106.592973,604.0,-0.218271,-0.226667,0.314675,7.536567e-06,-0.000017,0.000018
0-4389467,2023-06-23 01:56:34,3,38.346019,4.412427,LOW,0,0,d0,carthe_cnr_19,68919.839856,-89247.018689,599.0,-0.216404,-0.243979,0.326124,1.344408e-06,-0.000022,0.000022
0-4389467,2023-06-23 02:06:40,3,38.344624,4.410925,LOW,0,0,d0,carthe_cnr_19,68789.865565,-89402.998266,606.0,-0.216676,-0.253510,0.333491,-3.096268e-06,-0.000014,0.000014


In [96]:
dfs = pyn.drifters.smooth_all(df, "lowess", "30min", spectral_diff=False, geo=True)

In [99]:
dss = dfs.reset_index().set_index(["id"]).to_xarray()

In [98]:
DS = []
for id_ in dfs.id.unique():
    df_ = dfs[dfs.id == id_]
    ds_ = df_.to_xarray()
    ds_ = ds_.assign_coords(id=id_).expand_dims("id")
    DS.append(ds_)
dss = xr.concat(DS, dim="id")

In [100]:
dss

<xarray.Dataset>
Dimensions:   (id: 40507)
Coordinates:
  * id        (id) object '0-4388553' '0-4388553' ... '0-4389467' '0-4389467'
Data variables: (12/17)
    time      (id) datetime64[ns] 2023-04-23T21:55:10 ... 2023-06-23T02:10:40
    x         (id) float64 -3.04e+05 -3.04e+05 ... 6.913e+04 6.874e+04
    y         (id) float64 3.063e+05 3.063e+05 ... -8.903e+04 -8.947e+04
    u         (id) float64 0.2064 0.2064 0.1604 0.1288 ... -0.24 -0.2224 -0.2161
    v         (id) float64 0.1106 0.1106 0.1311 ... -0.2281 -0.2261 -0.2567
    U         (id) float64 0.2342 0.2342 0.2071 0.189 ... 0.3311 0.3172 0.3355
    ...        ...
    au        (id) float64 -1.278e-05 -1.278e-05 ... 6.633e-06 6.633e-06
    av        (id) float64 5.674e-06 5.674e-06 ... -7.926e-06 -7.926e-06
    Auv       (id) float64 1.399e-05 1.399e-05 2.287e-05 ... 1.034e-05 1.034e-05
    X         (id) float64 4.316e+05 4.316e+05 4.315e+05 ... 1.127e+05 1.128e+05
    gap_mask  (id) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
    gaps      (id) float64 0.0 11.0 8.0 0.0 -6.0 ... 256.0 253.0 249.0 240.0

In [91]:
dss.isel(id=0).u.dropna("time")

<xarray.DataArray 'u' (time: 2553)>
array([0.2063872 , 0.2063872 , 0.16036598, ..., 0.25966402, 0.23256958,
       0.23256958])
Coordinates:
    id       <U9 '0-4388553'
  * time     (time) datetime64[ns] 2023-04-23T21:55:10 ... 2023-06-16T01:55:10

In [37]:
dss.x.isel(id=0).dropna(dim="time")  # .plot()

<xarray.DataArray 'x' (time: 2553)>
array([-303988.8066492 , -303988.8066492 , -303660.97324825, ...,
        162245.30139746,  162688.70534107,  162688.70534107])
Coordinates:
  * time     (time) datetime64[ns] 2023-04-23T21:55:10 ... 2023-06-16T01:55:10
    id       <U9 '0-4388553'

In [36]:
dss.x.sel(id="0-4388553")  # .x.plot()

<xarray.DataArray 'x' (time: 40507)>
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
  * time     (time) datetime64[ns] 2023-04-23T18:03:46 ... 2023-06-26T05:32:21
    id       <U9 '0-4388553'